In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import os

In [3]:
os.chdir('/content/drive/MyDrive/EfficientDet2/')

#1. Zylo117

In [ ]:
# Install requirements

!pip install pycocotools numpy opencv-python tqdm tensorboard tensorboardX pyyaml webcolors
# !pip install torch==1.4.0
# !pip install torchvision==0.5.0

In [ ]:
# Download pretrain weights

!wget https://github.com/zylo117/Yet-Another-Efficient-Pytorch/releases/download/1.0/efficientdet-d0.pth

Create .yml file in projects directory
 
```
project_name: EfficientDet2
train_set: train
val_set: val
num_gpus: 1  # 0 means using cpu, 1-N means using gpus 

# mean and std in RGB order, actually this part should remain unchanged as long as your dataset is similar to coco.
mean: [0.485, 0.456, 0.406]
std: [0.229, 0.224, 0.225]

# this is coco anchors, change it if necessary
anchors_scales: '[2 ** 0, 2 ** (1.0 / 3.0), 2 ** (2.0 / 3.0)]'
anchors_ratios: '[(1.0, 1.0), (1.4, 0.7), (0.7, 1.4)]'

# objects from all labels from your dataset with the order from your annotations.
# its index must match your dataset's category_id.
# category_id is one_indexed,
# for example, index of 'car' here is 2, while category_id of is 3
obj_list: ['fracture']
```

In [12]:
# Train in xray image

!python train.py -c 1 -p EfficientDet2 --batch_size 8 --lr 1e-5 --num_epochs 200 --load_weights last

loading annotations into memory...
Done (t=0.00s)
creating index...
index created!
/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:477: UserWarning: This DataLoader will create 12 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))
loading annotations into memory...
Done (t=0.00s)
creating index...
index created!
using weights logs//EfficientDet2/efficientdet-d1_118_4522.pth
[Info] loaded weights: efficientdet-d1_118_4522.pth, resuming checkpoint from step: 4522
Step: 4559. Epoch: 119/200. Iteration: 38/38. Cls loss: 0.02397. Reg loss: 0.13585. Total loss: 0.15982: 100% 38/38 [02:44<00:00,  4.34s/it]
Val. Epoch: 119/200. Classification loss: 0.45522. Regression loss: 2.54749. Total l

In [16]:
!python coco_eval.py -p EfficientDet2 -c 1 -w /content/drive/MyDrive/EfficientDet2/logs/EfficientDet2/efficientdet-d1_118_4522.pth

running coco-style evaluation on project EfficientDet2, weights /content/drive/MyDrive/EfficientDet2/logs/EfficientDet2/efficientdet-d1_118_4522.pth...
loading annotations into memory...
Done (t=0.00s)
creating index...
index created!
100% 100/100 [00:05<00:00, 17.32it/s]
Loading and preparing results...
DONE (t=0.01s)
creating index...
index created!
BBox
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=0.10s).
Accumulating evaluation results...
DONE (t=0.02s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.272
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 0.657
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 0.120
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = 0.116
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = 0.278
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=100 ] = 0.334
 Average Recall     (AR) @[ Io

backbone.py		     efficientnet  res
benchmark		     LICENSE	   test
coco_eval.py		     logs	   train.py
datasets		     pretrain	   tutorial
efficientdet		     projects	   utils
efficientdet_test.py	     __pycache__   val2017_bbox_results.json
efficientdet_test_videos.py  readme.md	   val_bbox_results.json


In [11]:
# Prediction
import torch
from torch.backends import cudnn

from backbone import EfficientDetBackbone
import cv2
import matplotlib.pyplot as plt
import numpy as np
import glob
import random

from efficientdet.utils import BBoxTransform, ClipBoxes
from utils.utils import preprocess, invert_affine, postprocess

compound_coef = 1
id = 0
force_input_size = None  # set None to use default size

for img_path in glob.glob('/content/drive/MyDrive/EfficientDet2/datasets/EfficientDet2/val/*.jpg'):
  id += 1
  threshold = 0.25
  iou_threshold = 0.2

  use_cuda = True
  use_float16 = False
  cudnn.fastest = True
  cudnn.benchmark = True

  obj_list = ['fracture']

  # tf bilinear interpolation is different from any other's, just make do
  input_sizes = [512, 640, 768, 896, 1024, 1280, 1280, 1536]
  input_size = input_sizes[compound_coef] if force_input_size is None else force_input_size
  ori_imgs, framed_imgs, framed_metas = preprocess(img_path, max_size=input_size)

  if use_cuda:
      x = torch.stack([torch.from_numpy(fi).cuda() for fi in framed_imgs], 0)
  else:
      x = torch.stack([torch.from_numpy(fi) for fi in framed_imgs], 0)

  x = x.to(torch.float32 if not use_float16 else torch.float16).permute(0, 3, 1, 2)

  model = EfficientDetBackbone(compound_coef=compound_coef, num_classes=len(obj_list),

                               # replace this part with your project's anchor config
                               ratios=[(0.7, 1.4), (1.0, 1.0), (1.5, 0.7)],
                               scales=[2 ** 0, 2 ** (1.0 / 3.0), 2 ** (2.0 / 3.0)])

  model.load_state_dict(torch.load('/content/drive/MyDrive/EfficientDet2/logs/EfficientDet2/efficientdet-d1_118_4522.pth'))
  model.requires_grad_(False)
  model.eval()

  if use_cuda:
      model = model.cuda()
  if use_float16:
      model = model.half()

  with torch.no_grad():
      features, regression, classification, anchors = model(x)

      regressBoxes = BBoxTransform()
      clipBoxes = ClipBoxes()

      out = postprocess(x,
                        anchors, regression, classification,
                        regressBoxes, clipBoxes,
                        threshold, iou_threshold)

  out = invert_affine(framed_metas, out)
  image = ori_imgs[0]
  for i in range(len(ori_imgs)):
      if len(out[i]['rois']) == 0:
          continue
      ori_imgs[i] = ori_imgs[i].copy()
      for j in range(len(out[i]['rois'])):
          (x1, y1, x2, y2) = out[i]['rois'][j].astype(np.int)
          cv2.rectangle(ori_imgs[i], (x1, y1), (x2, y2), (255, 255, 0), 2)
          obj = obj_list[out[i]['class_ids'][j]]
          score = float(out[i]['scores'][j])

          image = cv2.putText(ori_imgs[i], '{}, {:.3f}'.format(obj, score),
                      (x1, y1 + 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5,
                      (255, 255, 0), 1)

  cv2.imwrite(f'res/{id}.jpg', image)
  print(f'image {id}: Saved!')
    # plt.figure(figsize=(7, 7))
    # plt.imshow(image)
    # plt.savefig('res/1.jpg')


image 1: Saved!
image 2: Saved!
image 3: Saved!
image 4: Saved!
image 5: Saved!
image 6: Saved!
image 7: Saved!
image 8: Saved!
image 9: Saved!
image 10: Saved!
image 11: Saved!
image 12: Saved!
image 13: Saved!
image 14: Saved!
image 15: Saved!
image 16: Saved!
image 17: Saved!
image 18: Saved!
image 19: Saved!
image 20: Saved!
image 21: Saved!
image 22: Saved!
image 23: Saved!
image 24: Saved!
image 25: Saved!
image 26: Saved!
image 27: Saved!
image 28: Saved!
image 29: Saved!
image 30: Saved!
image 31: Saved!
image 32: Saved!
image 33: Saved!
image 34: Saved!
image 35: Saved!
image 36: Saved!
image 37: Saved!
image 38: Saved!
image 39: Saved!
image 40: Saved!
image 41: Saved!
image 42: Saved!
image 43: Saved!
image 44: Saved!
image 45: Saved!
image 46: Saved!
image 47: Saved!
image 48: Saved!
image 49: Saved!
image 50: Saved!
image 51: Saved!
image 52: Saved!
image 53: Saved!
image 54: Saved!
image 55: Saved!
image 56: Saved!
image 57: Saved!
image 58: Saved!
image 59: Saved!
image 

#2. Roboflow

In [ ]:
%cd /content/drive/MyDrive

/content/drive/MyDrive


In [ ]:
! git clone https://github.com/roboflow-ai/Monk_Object_Detection.git

fatal: destination path 'Monk_Object_Detection' already exists and is not an empty directory.


In [ ]:
! cd Monk_Object_Detection/3_mxrcnn/installation && cat requirements_colab.txt | xargs -n 1 -L 1 pip install

ERROR: Operation cancelled by user
^C


In [ ]:
#fixed version of tqdm output for Colab
!pip install --force https://github.com/chengs/tqdm/archive/colab.zip
#IGNORE restart runtime warning, it is indeed installed
#missing a few extra packages that we will need later! 
!pip install efficientnet_pytorch
!pip install tensorboardX

  Using cached https://github.com/chengs/tqdm/archive/colab.zip
  Created wheel for tqdm: filename=tqdm-4.28.1-py2.py3-none-any.whl size=47868 sha256=4a812857873069094825f4da4ac043677fe4f5e32c5804b2dc8816dcd5f10f30
  Stored in directory: /tmp/pip-ephem-wheel-cache-fnqcqmty/wheels/41/18/ee/d5dd158441b27965855b1bbae03fa2d8a91fe645c01b419896
Successfully built tqdm
ERROR: Operation cancelled by user


In [ ]:
%mkdir Xray/

mkdir: cannot create directory ‘Xray/’: File exists


In [ ]:
!mkdir Xray/annotations
!mkdir Xray/Annotations
!mkdir Xray/Images

In [ ]:
%cp train/_annotations.coco.json Xray/annotations/instances_Images.json

cp: cannot stat 'train/_annotations.coco.json': No such file or directory


In [ ]:
%cp train/*.jpg Xray/Images/

cp: cannot stat 'train/*.jpg': No such file or directory


In [ ]:
import os
import sys
sys.path.append("Monk_Object_Detection/4_efficientdet/lib/");

In [ ]:
from train_detector import Detector

In [ ]:
gtf = Detector();

In [ ]:
#directs the model towards file structure
root_dir = "./";
coco_dir = "Xray";
img_dir = "./";
set_dir = "Images";

In [ ]:
#smells like some free compute from Colab, nice
gtf.Train_Dataset(root_dir, coco_dir, img_dir, set_dir, batch_size=8, image_size=512, use_gpu=True)

loading annotations into memory...
Done (t=0.00s)
creating index...
index created!


/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:477: UserWarning: This DataLoader will create 3 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


In [ ]:
gtf.Model();

Loaded pretrained weights for efficientnet-b0


In [ ]:
gtf.Set_Hyperparams(lr=0.0001, val_interval=1, es_min_delta=0.0, es_patience=0)

In [ ]:
%%time
gtf.Train(num_epochs=20, model_output_dir="trained/");

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:477: UserWarning: This DataLoader will create 3 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


KeyboardInterrupt: ignored

In [ ]:
import os
import sys
sys.path.append("Monk_Object_Detection/4_efficientdet/lib/");

In [ ]:
from infer_detector import Infer

In [ ]:
gtf = Infer();

In [ ]:
#our trained model weights are in here in onxx format
gtf.Model(model_dir="trained/")

In [ ]:
#extract class list from our annotations
import json
with open('Xray/train/_annotations.coco.json') as json_file:
    data = json.load(json_file)
class_list = []
for category in data['categories']:
  class_list.append(category['name'])

In [ ]:
%%time
test_images = [f for f in os.listdir('Xray/test') if f.endswith('.jpg')]
# import random
# img_path = "Xray/test/" + random.choice(test_images);

for img_path in test_images:
  img_path = 'Xray/test/' + img_path
  duration, scores, labels, boxes = gtf.Predict(img_path, class_list, vis_threshold=0.2);

In [ ]:
from IPython.display import Image
Image(filename='output.jpg') 

#3. Signatrix

In [ ]:
!!curl -L "https://app.roboflow.com/ds/4q0xWQkGJr?key=fYtlCsl92J" > roboflow.zip; unzip roboflow.zip; rm roboflow.zip

In [ ]:
!git clone https://github.com/signatrix/efficientdet.git

Cloning into 'efficientdet'...
remote: Enumerating objects: 144, done.
remote: Total 144 (delta 0), reused 0 (delta 0), pack-reused 144
Receiving objects: 100% (144/144), 28.03 MiB | 15.73 MiB/s, done.
Resolving deltas: 100% (68/68), done.


In [ ]:
!python train.py --lr 1e-5 --batch_size 8 --num_epochs 100 --image_size 416